In [1]:
# %pip uninstall protobuf
%pip install protobuf==3.20

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unbabel-comet 2.2.2 requires protobuf<5.0.0,>=4.24.4, but you have protobuf 3.20.0 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery 3.10.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery-conn

In [2]:
import os
import random
import torch
import numpy as np
from tqdm import tqdm
from typing import Dict, List

from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    AutoModelForSequenceClassification,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

from peft import (
    LoraConfig, 
    prepare_model_for_kbit_training, 
    get_peft_model
)

from comet import (
    download_model, 
    load_from_checkpoint
)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-10-17 06:34:59.582289: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 06:35:04.747310: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
!ls checkpoints/xalma-lora-3.0-wmt

checkpoint-100
checkpoint-200
checkpoint-300
checkpoint-400
checkpoint-500
checkpoint-600
checkpoint-700
checkpoint-749


In [4]:
peft_model_id = f"./checkpoints/xalma-lora-3.0-wmt/checkpoint-749"
model = AutoModelForCausalLM.from_pretrained(peft_model_id)

Loading checkpoint shards: 100%|██████████| 6/6 [08:46<00:00, 87.70s/it] 


In [5]:
def seed_everything(seed):
    # Fix seeds
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(123456)

In [6]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(
            in_features=5120, out_features=5120, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=5120, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=5120, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear(
            in_features=5120, out_features=5120, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
      

In [9]:
dataset = load_dataset(
    "json",
    data_files={
        "val": "/home/jupyter/datasphere/project/data/flores200_devtest/en_uz_devtest.jsonl",
    },
)

In [10]:
model_path = "haoranxu/X-ALMA-13B-Pretrain"
tokenizer = AutoTokenizer.from_pretrained(
    model_path, model_max_length=256, padding_side="left"
)

In [11]:
def tokenize(samples):
    inputs = [
        f"Translate this from Uzbek to English:\nUzbek: {uz}\nEnglish:"
        for uz in samples["uz"]
    ]
    targets = samples["en"]

    model_inputs = tokenizer(
        inputs,
        truncation=True,
        max_length=256,
        padding="max_length",
        return_tensors="pt",
    )
    labels = tokenizer(
        targets,
        truncation=True,
        max_length=256,
        padding="max_length",
        return_tensors="pt",
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
tokenized_val_dataset = dataset["val"].map(tokenize, batched=True)

Map: 100%|██████████| 1012/1012 [00:08<00:00, 114.80 examples/s]


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_path_comet = download_model("Unbabel/wmt22-comet-da")
model_comet = load_from_checkpoint(model_path_comet)
model_comet = model_comet.to(device)

def comet(data: List[Dict[str, str]]) -> List[float]:
    
    '''Format
    data = [
    {
        # Source, текст, который надо перевести, src
        "src": "В понедельник", 
        
        # Machine Translation
        "mt": "On Monday", 
        
        # Эталонный перевод, en
        "ref": "On Monday" 
    }'''
    
    
    comet_metric = model_comet.predict(data, batch_size=8, gpus=1)
    return comet_metric.scores


Fetching 5 files: 100%|██████████| 5/5 [00:22<00:00,  4.47s/it]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../tmp/xdg_cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
Encoder model frozen.
/home/jupyter/.local/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [14]:
all_predictions = []
all_references = []
all_sources = []

# batch_size = 1      # in Kaggle
batch_size = 4    # in DataSphere

val_dataset = tokenized_val_dataset.shuffle().select(range(100))

model.eval()
model.to(device)

for i in tqdm(range(0, len(val_dataset), batch_size)):
    batch = val_dataset.select(range(i, min(i + batch_size, len(val_dataset))))
    input_ids = torch.tensor(batch["input_ids"]).to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=input_ids,
            num_beams=5,
            max_new_tokens=200,
            no_repeat_ngram_size=3,
            early_stopping=True,
        ).to(device)
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    for uz, en, translated in zip(batch["uz"], batch["en"], outputs):
        all_sources.append(uz)
        all_references.append(en)
        all_predictions.append(translated.split('English:')[-1].strip())

100%|██████████| 25/25 [20:56<00:00, 50.25s/it]  


In [15]:
def evaluate(model):
    model.eval()

    comet_data = [
        {"src": src, "mt": pred, "ref": ref}
        for src, pred, ref in zip(all_sources, all_predictions, all_references)
    ]

    comet_scores = comet(comet_data)
    avg_comet_score = sum(comet_scores) / len(comet_scores)

    return avg_comet_score, all_predictions, all_references

avg_comet_score, predictions, references = evaluate(model, tokenizer)
print(f"Average COMET score: {avg_comet_score:.4f}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable

Average COMET score: 0.4237
